## Pandas 技巧

In [1]:
import pandas as pd

In [2]:
# data
locations = pd.read_csv('./data/locations.gz',sep='\t',header=None, 
                        dtype={'latitude':pd.np.float32,'longitude':pd.np.float32,'duration':pd.np.int},
                        names=['phone','latitude','longitude','datetime','duration'])
locations.head()

,phone,latitude,longitude,datetime,duration
0,00050C1891C1DBF6C9ECA6ADED2BE6DE,19.289000,-99.005997,15/10/2014 22:10:41,48
1,00050C1891C1DBF6C9ECA6ADED2BE6DE,19.289000,-99.005997,15/10/2014 21:09:28,78
2,0006EBC091C7E79FA9BF9164528929F8,19.463314,-99.146431,15/10/2014 13:01:33,22
3,0006EBC091C7E79FA9BF9164528929F8,19.471701,-99.140297,15/10/2014 07:07:51,29
4,0006EBC091C7E79FA9BF9164528929F8,19.463314,-99.146431,15/10/2014 12:12:09,26


### 将字符串存储的日期变成日期object

In [3]:
locations['datetime'] = pd.to_datetime(locations['datetime'],format="%d/%m/%Y %H:%M:%S")
locations['datetime'].head()

0   2014-10-15 22:10:41
1   2014-10-15 21:09:28
2   2014-10-15 13:01:33
3   2014-10-15 07:07:51
4   2014-10-15 12:12:09
Name: datetime, dtype: datetime64[ns]

### 按给定时间段resample并填充

对于每个用户
1. 合并30分钟之内的duration并求和
2. 如果某个30分钟时间段没有duration那么该时间段不会出现，对这些时间段，填充0

In [4]:
sub = locations.groupby('phone').get_group('000A8481F9274EEA3F994376973B23CE')

# add DateTimeIndex
samples = sub.set_index('datetime')['duration'].resample('30T').sum()
samples

datetime
2014-10-15 16:00:00    122.0
2014-10-15 16:30:00      NaN
2014-10-15 17:00:00      NaN
2014-10-15 17:30:00      NaN
2014-10-15 18:00:00    113.0
2014-10-15 18:30:00      NaN
2014-10-15 19:00:00    441.0
2014-10-15 19:30:00      NaN
2014-10-15 20:00:00    411.0
Freq: 30T, Name: duration, dtype: float64

1. 结果里面有NaN值
2. 时间段不完整，只有16:00 到 20:00
3. 需要填充一天的所有时间段

In [5]:
samples = samples.fillna(0.0)
ranges = pd.date_range('15/10/2014',freq='30T',periods=48)
samples = samples.reindex(ranges).fillna(0)
samples

2014-10-15 00:00:00      0.0
2014-10-15 00:30:00      0.0
2014-10-15 01:00:00      0.0
2014-10-15 01:30:00      0.0
2014-10-15 02:00:00      0.0
2014-10-15 02:30:00      0.0
2014-10-15 03:00:00      0.0
2014-10-15 03:30:00      0.0
2014-10-15 04:00:00      0.0
2014-10-15 04:30:00      0.0
2014-10-15 05:00:00      0.0
2014-10-15 05:30:00      0.0
2014-10-15 06:00:00      0.0
2014-10-15 06:30:00      0.0
2014-10-15 07:00:00      0.0
2014-10-15 07:30:00      0.0
2014-10-15 08:00:00      0.0
2014-10-15 08:30:00      0.0
2014-10-15 09:00:00      0.0
2014-10-15 09:30:00      0.0
2014-10-15 10:00:00      0.0
2014-10-15 10:30:00      0.0
2014-10-15 11:00:00      0.0
2014-10-15 11:30:00      0.0
2014-10-15 12:00:00      0.0
2014-10-15 12:30:00      0.0
2014-10-15 13:00:00      0.0
2014-10-15 13:30:00      0.0
2014-10-15 14:00:00      0.0
2014-10-15 14:30:00      0.0
2014-10-15 15:00:00      0.0
2014-10-15 15:30:00      0.0
2014-10-15 16:00:00    122.0
2014-10-15 16:30:00      0.0
2014-10-15 17:

### 获得数据里面的全部日期

In [6]:
locations['datetime'].map(lambda dt:dt.date()).unique()

array([datetime.date(2014, 10, 15)], dtype=object)

### 数据过滤的手段

In [7]:
locations=locations[locations['duration'].notnull()]
locations_filtered = locations[locations['duration']>10]
locations_filtered.head()

,phone,latitude,longitude,datetime,duration
0,00050C1891C1DBF6C9ECA6ADED2BE6DE,19.289000,-99.005997,2014-10-15 22:10:41,48
1,00050C1891C1DBF6C9ECA6ADED2BE6DE,19.289000,-99.005997,2014-10-15 21:09:28,78
2,0006EBC091C7E79FA9BF9164528929F8,19.463314,-99.146431,2014-10-15 13:01:33,22
3,0006EBC091C7E79FA9BF9164528929F8,19.471701,-99.140297,2014-10-15 07:07:51,29
4,0006EBC091C7E79FA9BF9164528929F8,19.463314,-99.146431,2014-10-15 12:12:09,26


### 横向批量增加行

In [8]:
columns =  locations.columns
coloumns_new = columns.append(pd.Index(['new_col','new_col1']))
locations_new = locations.reindex_axis(coloumns_new,axis='columns')
locations_new.head()

,phone,latitude,longitude,datetime,duration,new_col,new_col1
0,00050C1891C1DBF6C9ECA6ADED2BE6DE,19.289000,-99.005997,2014-10-15 22:10:41,48,NaN,NaN
1,00050C1891C1DBF6C9ECA6ADED2BE6DE,19.289000,-99.005997,2014-10-15 21:09:28,78,NaN,NaN
2,0006EBC091C7E79FA9BF9164528929F8,19.463314,-99.146431,2014-10-15 13:01:33,22,NaN,NaN
3,0006EBC091C7E79FA9BF9164528929F8,19.471701,-99.140297,2014-10-15 07:07:51,29,NaN,NaN
4,0006EBC091C7E79FA9BF9164528929F8,19.463314,-99.146431,2014-10-15 12:12:09,26,NaN,NaN


### 指定格式节省空间

In [9]:
v1 = pd.read_csv('./data/volumes.gz',index_col=0)
v2 = pd.read_csv('./data/volumes.gz',index_col=0,dtype='uint32',converters={0:str})

musage_v1 = v1.memory_usage().sum()*1.0
musage_v2 = v2.memory_usage().sum()*1.0

print "Memory Usage"
print "Default:",musage_v1
print "Use uint32:",musage_v2
print "Saved: %.2f%%"%(100.0*(musage_v1-musage_v2)/musage_v1)

Memory Usage
Default: 107497728.0
Use uint32: 54302976.0
Saved: 49.48%


### pandas.DataFrame.add 的另外一种形式
不会强制将类型转化为float64

In [10]:
t1 = pd.DataFrame([(1,2),(3,4),(5,6)], columns=["A","B"],dtype='uint32')
t2 = pd.DataFrame([(1,2),(3,4),(5,6)], columns=["D","C"],dtype='uint32')
# begin
t3 = t1
t3 = t3.reindex_axis(t1.index.union(t2.index),copy=False,fill_value=pd.np.uint32(0))
t3 = t3.reindex_axis(t1.columns.union(t2.columns),axis='columns',copy=False,fill_value=pd.np.uint32(0))
t3.loc[t2.index,t2.columns] += t2
# end
t3.dtypes
t3

,A,B,C,D
0,1,2,2,1
1,3,4,4,3
2,5,6,6,5


### 横向数据扩充

In [19]:
dt = pd.read_csv('./data/volumes3.gz',index_col=0,dtype='uint32',converters={0:str})
# select row
row = dt.ix[0]
row.index = pd.DatetimeIndex(row.index)
dates = pd.date_range(row.index.min(),row.index.max(),freq='30T')
# print row.dtype
row = row.reindex(dates,fill_value=0)
# print row.dtype
# row.index.max()
dt.columns = pd.DatetimeIndex(dt.columns)
dates = pd.date_range(dt.columns.min(),dt.columns.max(),freq='30T')
dt.reindex_axis(dates,axis='columns',fill_value=0)


,2014-10-01 00:00:00,2014-10-01 00:30:00,2014-10-01 01:00:00,2014-10-01 01:30:00,2014-10-01 02:00:00,2014-10-01 02:30:00,2014-10-01 03:00:00,2014-10-01 03:30:00,2014-10-01 04:00:00,2014-10-01 04:30:00,...,2014-12-31 19:00:00,2014-12-31 19:30:00,2014-12-31 20:00:00,2014-12-31 20:30:00,2014-12-31 21:00:00,2014-12-31 21:30:00,2014-12-31 22:00:00,2014-12-31 22:30:00,2014-12-31 23:00:00,2014-12-31 23:30:00
000011F2BE4C5E7D456026CD791185B3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00007CD8FD986A1275746E8FFB83A483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000F50CD5823204AF3F8A479B10EA9A,0,0,0,0,0,0,0,0,0,0,...,0,0,7227,0,0,0,0,1009,0,0
0001AEF99C0AC025FC20329B9C2467AC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,123,0
0002159CB3335AE65713EE8A74FD8B72,0,0,0,0,0,0,0,0,0,0,...,65032,0,0,0,0,0,98,0,0,0
0002C2C385D238CA11D412F6660D7064,0,0,0,0,0,0,0,0,0,0,...,871,0,0,0,0,2062,0,0,0,972
00037ED41AC68CB05CDD8D24C09DC67D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4514,0,0,68,0,0,0
0005F8B895637FF40E5321461EF8DB31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,334,0
000621483722CA8E9052A9FB6E282C5D,0,0,0,0,0,0,0,0,0,0,...,243,0,0,0,0,0,0,0,241,0
00067C78A90496F452BA29D084B61FA9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6132,0,0,0,0


### DataFrame ApplyMap

In [23]:
import math
dt = pd.read_csv('./data/volumes3.gz',index_col=0,dtype='uint32',converters={0:str})
def volume2status(vol):
    return 0 if vol == 0 else int(math.ceil(math.log10(vol)))
dt = dt.applymap(volume2status)
dt.dtypes

2014-10-01 00:00:00    int64
2014-10-01 00:30:00    int64
2014-10-01 01:00:00    int64
2014-10-01 01:30:00    int64
2014-10-01 02:00:00    int64
2014-10-01 02:30:00    int64
2014-10-01 03:00:00    int64
2014-10-01 03:30:00    int64
2014-10-01 04:00:00    int64
2014-10-01 04:30:00    int64
2014-10-01 05:00:00    int64
2014-10-01 05:30:00    int64
2014-10-01 06:00:00    int64
2014-10-01 06:30:00    int64
2014-10-01 07:00:00    int64
2014-10-01 07:30:00    int64
2014-10-01 08:00:00    int64
2014-10-01 08:30:00    int64
2014-10-01 09:00:00    int64
2014-10-01 09:30:00    int64
2014-10-01 10:00:00    int64
2014-10-01 10:30:00    int64
2014-10-01 11:00:00    int64
2014-10-01 11:30:00    int64
2014-10-01 12:00:00    int64
2014-10-01 12:30:00    int64
2014-10-01 13:00:00    int64
2014-10-01 13:30:00    int64
2014-10-01 14:00:00    int64
2014-10-01 14:30:00    int64
                       ...  
2014-12-31 09:00:00    int64
2014-12-31 09:30:00    int64
2014-12-31 10:00:00    int64
2014-12-31 10:

In [41]:
dt = pd.read_csv('./data/volumes3.gz',index_col=0,dtype='uint32',converters={0:str})
dt = dt[dt.sum(axis=1)!=0]
dt.sum(axis=1)

000011F2BE4C5E7D456026CD791185B3     348091
00007CD8FD986A1275746E8FFB83A483     707225
0000F50CD5823204AF3F8A479B10EA9A    2488788
0001AEF99C0AC025FC20329B9C2467AC    3029437
0002159CB3335AE65713EE8A74FD8B72    6139952
0002C2C385D238CA11D412F6660D7064    1494942
00037ED41AC68CB05CDD8D24C09DC67D    2738605
0005F8B895637FF40E5321461EF8DB31    2979790
000621483722CA8E9052A9FB6E282C5D     665473
00067C78A90496F452BA29D084B61FA9    4139711
000697293F93FC8FC491671E95BA6D94    2013491
000722192641E81C5F6C7E9366A9FABB    4139696
00076C783BA538C5EBA97B4B225446EE    2978876
0008192F9D3E24572C9CF5FF917C92A1     393907
000851BA098810EA13D7FF89BE4BC452    1231624
0008FDDAEAADF73833BD507C61754B9C     280823
0009038503E928E203232446BF2E3E67    3202025
0009308C08A5A2BFACC20C2540A7AF36      15775
000AB17A0A9994663210F6EF72353507     507893
000B2CB67039B92F90208E9DDEEF8F09     796909
000BA7194C04187DDF00E07C5500E50A    1922638
000C66FF3CE79C40610F56807F43E33B     837498
000DF5A2348928C3D93A3EDE2173E882